In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import requests
import json
import weaviate

from transformers import AutoModel, AutoTokenizer
import torch
import weaviate.classes as wvc
import numpy as np


In [4]:
llm_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b")
llm_model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")
llm_model.eval()

/opt/homebrew/anaconda3/envs/mlops-auth-api/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True

In [6]:
embedding_model_name = "BAAI/bge-m3"  # 모델 이름
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
embedding_model = AutoModel.from_pretrained(embedding_model_name)
embedding_model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementw

In [46]:
def generate_with_context(context, question, model=llm_model, tokenizer=llm_tokenizer, max_length=100):
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")

    # 모델을 사용해 답변 생성
    attention_mask = inputs["attention_mask"]
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=attention_mask, 
        max_length=max_length, 
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [8]:
def do_embedding(sentences: list, model=embedding_model, tokenizer=embedding_tokenizer):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings[0]

In [9]:
client = weaviate.connect_to_local(
    host="127.0.0.1",  # Use a string to specify the host
    port=8080,
    grpc_port=50051,
)

In [10]:
client.collections.delete("ETF")
my_vecs = client.collections.create(
        "ETF",
        vectorizer_config=wvc.config.Configure.Vectorizer.none(),
        vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
            distance_metric=wvc.config.VectorDistances.COSINE # select prefered distance metric
        ),
    )

In [11]:
etf_contexts = [
    "ETF(Exchange Traded Fund)는 주식처럼 거래소에서 매매되는 펀드입니다. ETF는 주식, 채권, 원자재 등의 자산을 기초로 하며, 분산 투자를 통해 리스크를 줄이는 데 효과적입니다.",
    "S&P 500 ETF는 미국의 S&P 500 지수를 추종하는 대표적인 ETF입니다. 이 ETF는 미국 대형주에 분산 투자하는 효과를 제공하며, 장기적으로 안정적인 수익을 추구하는 투자자에게 인기가 많습니다.",
    "섹터 ETF는 특정 산업 분야에 집중 투자하는 ETF입니다. 기술, 헬스케어, 에너지 등 다양한 섹터에 투자할 수 있으며, 특정 산업의 성장 가능성에 베팅하려는 투자자들에게 적합합니다.",
    "레버리지 ETF는 기초 자산의 수익률을 2배 또는 3배로 추적하도록 설계된 ETF입니다. 이는 고수익을 기대할 수 있지만, 동시에 큰 손실 위험도 동반합니다. 단기 투자자에게 적합한 상품입니다.",
    "배당주 ETF는 배당 수익을 중시하는 투자자들에게 적합한 ETF입니다. 고배당 주식들로 구성되어 정기적인 배당 수익을 제공하며, 장기적인 인컴 수익을 추구할 수 있습니다.",
    "채권 ETF는 국채, 회사채, 지방채 등 다양한 채권에 투자하는 ETF입니다. 주식보다 안정적인 수익을 제공하며, 포트폴리오의 위험을 분산하기 위한 수단으로 많이 활용됩니다.",
    "ESG ETF는 환경, 사회, 지배구조(ESG) 기준을 충족하는 기업들로 구성된 ETF입니다. 지속 가능한 투자 전략을 추구하는 투자자들에게 인기가 있으며, 장기적인 사회적 책임과 수익을 함께 고려합니다.",
    "글로벌 ETF는 전 세계 다양한 시장에 분산 투자하는 ETF입니다. 미국, 유럽, 아시아 등 다양한 지역의 주식과 채권에 투자하며, 글로벌 분산 투자를 통해 위험을 줄이는 것이 목적입니다.",
    "커버드 콜 ETF는 주식에 대한 콜 옵션을 판매하는 전략을 사용하여 수익을 창출하는 ETF입니다. 이 전략은 시장 변동성이 낮을 때 꾸준한 수익을 제공하며, 위험을 통제하는 데도 효과적입니다.",
    "원자재 ETF는 금, 은, 석유, 농산물 등 원자재 가격을 추적하는 ETF입니다. 원자재 시장에 직접 투자하기 어려운 개인 투자자에게 대안이 될 수 있으며, 인플레이션 해지(hedge) 수단으로도 사용됩니다.",
    "테크놀로지 섹터 ETF는 IT 및 기술 관련 기업에 집중 투자하는 ETF입니다. 애플, 마이크로소프트, 구글과 같은 주요 기술 기업들이 포함되어 있으며, 기술 혁신과 성장에 투자할 기회를 제공합니다.",
    "소형주 ETF는 소형 기업들에 투자하는 ETF입니다. 성장 잠재력이 높은 기업들에 대한 투자를 통해 높은 수익률을 기대할 수 있지만, 변동성이 큰 점도 고려해야 합니다.",
    "부동산 ETF는 리츠(REITs)와 같은 부동산 관련 자산에 투자하는 ETF입니다. 안정적인 배당 수익을 제공하며, 장기적인 부동산 자산 증가에 대한 노출을 제공합니다.",
    "헬스케어 ETF는 제약, 바이오테크, 의료 서비스 등 헬스케어 분야에 집중 투자하는 ETF입니다. 헬스케어 산업의 성장 가능성에 베팅하려는 투자자들에게 적합합니다.",
    "에너지 ETF는 석유, 가스, 전력 등의 에너지 관련 자산에 투자하는 ETF입니다. 에너지 가격 변동에 민감하게 반응하며, 인플레이션 시기에 수익을 기대할 수 있습니다.",
    "인프라 ETF는 도로, 철도, 통신망 등 인프라 관련 자산에 투자하는 ETF입니다. 안정적인 수익과 함께 장기적인 인프라 개발의 혜택을 누릴 수 있는 투자 옵션입니다.",
    "금 ETF는 금의 시장 가격을 추적하는 ETF입니다. 금은 전통적으로 안전자산으로 여겨지며, 금융 위기나 인플레이션 시기에 인기를 끌곤 합니다.",
    "글로벌 채권 ETF는 전 세계의 국채와 회사채에 투자하는 ETF입니다. 다양한 국가와 기업의 채권을 포함해, 글로벌 분산 채권 투자를 통해 안정적인 수익을 제공합니다.",
    "디지털 자산 ETF는 비트코인, 이더리움과 같은 암호화폐에 투자하는 ETF입니다. 암호화폐의 변동성을 활용해 수익을 추구하지만, 높은 리스크도 동반됩니다.",
    "멀티자산 ETF는 주식, 채권, 원자재 등 여러 자산에 동시에 투자하는 ETF입니다. 포트폴리오 분산 효과를 극대화하며, 다양한 자산의 성과를 통해 위험을 관리할 수 있습니다."
]

In [15]:
etf = client.collections.get("ETF")
contexts = list()
for context in etf_contexts:
    contexts.append(wvc.data.DataObject(
        properties={"context":context}, 
        vector=do_embedding(context).numpy().tolist()
    ))
etf.data.insert_many(contexts)

BatchObjectReturn(_all_responses=[UUID('af30efeb-50c6-4a82-80d9-5587d69d245c'), UUID('ac86dc89-be06-40da-bfcc-885d0a68eda3'), UUID('ebf875ed-64a2-48ae-9a98-670b1c300f03'), UUID('a819bf8e-d912-41ba-b872-95be28712f3c'), UUID('62ddd63f-8481-403b-9a40-fd279a52324d'), UUID('5a356ce6-511b-41be-9e3b-9d6ef5726e19'), UUID('48264a8e-2b8f-498d-9a0e-4a3d5ab83680'), UUID('c3340454-37e1-4d7b-9ee4-8300f452871b'), UUID('2b5f5caf-ac25-4561-8a91-a2b4e8bea9bc'), UUID('d9f2d0c8-9d12-4541-a8bc-8247b5c3248d'), UUID('c3642315-747b-4c48-9cba-0573dd19793c'), UUID('c1a7505a-ee0d-4837-8cc8-6ce05e2b2c0d'), UUID('4cbbdf58-4e71-4645-978e-ae208ca30372'), UUID('bddb8f17-d03b-4bcd-a098-7d890597b89b'), UUID('56b7b146-7f1a-484b-8cea-a87c2f2d5d3e'), UUID('2a8e9fcd-0ffb-4d9d-9375-f4c4b864f756'), UUID('8596816e-6d6c-4d26-ad07-0804bd10b33d'), UUID('0b9f68ec-c147-4929-b08e-1a92b22fb11c'), UUID('74e9093a-dcd2-4bd7-9b65-643daeec35e7'), UUID('dfe14c39-83d7-4932-a3de-47320f1a355d')], elapsed_seconds=0.02007603645324707, errors={

In [18]:

etf = client.collections.get("ETF")
query_vector = do_embedding(question).numpy().tolist()
resps = etf.query.near_vector(
    near_vector=query_vector,
    limit=1,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

for obj in resps.objects:
    print(obj.properties)

{'context': '배당주 ETF는 배당 수익을 중시하는 투자자들에게 적합한 ETF입니다. 고배당 주식들로 구성되어 정기적인 배당 수익을 제공하며, 장기적인 인컴 수익을 추구할 수 있습니다.'}


In [21]:
def vector_search(question):
    etf = client.collections.get("ETF")
    query_vector = do_embedding(question).numpy().tolist()
    resps = etf.query.near_vector(
        near_vector=query_vector,
        limit=1,
        return_metadata=wvc.query.MetadataQuery(certainty=True)
    )
    return resps.objects[0].properties['context']


In [29]:
def rag(question, max_length=100):
    context = vector_search(question)
    return generate_with_context(context, question, max_length=max_length)

In [53]:
question = "레버리지 ETF에 대해 설명해줘"

rag(question, max_length=150)

'Context: 레버리지 ETF는 기초 자산의 수익률을 2배 또는 3배로 추적하도록 설계된 ETF입니다. 이는 고수익을 기대할 수 있지만, 동시에 큰 손실 위험도 동반합니다. 단기 투자자에게 적합한 상품입니다.\nQuestion: 레버리지 ETF에 대해 설명해줘\nAnswer: 레버리지 ETF는 기초자산의 수익률을 2배 또는 3배로 추적하도록 설계되었습니다. 따라서, 투자자는 기초 자산이 상승하면 더 많은 이익을 얻고 하락 시 손실을 보게 됩니다. 그러나, 이러한 레버리지 ETF가 장기적으로 시장보다 높은 성과를 달성한다는 보장은 없으며, 일부 펀드에서는 레버리지'